In [31]:
import json
import importlib
import os
current_dir=os.getcwd()
os.chdir("../../")
# print(os.getcwd())
import dist_utilities as dist_ult
# import noise_est_utilities as ne_ult
importlib.reload(dist_ult)
# importlib.reload(ne_ult)
os.chdir(current_dir)
import pickle
from qiskit.result import Result
from qiskit.visualization import plot_histogram, plot_distribution
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
from qiskit.result import QuasiDistribution

In [32]:
with open("ideal_counts_sim.json", "r") as f:
    ideal_counts=json.load(f)
with open("noisy_counts_sim.json", "r") as f:
    noisy_counts=json.load(f)
with open("noisy_noise_est_dist_sim.json", "r") as f:
    noisy_noise_est_dist=json.load(f)
with open("ideal_noise_est_counts_sim.json", "r") as f:
    ideal_noise_est_counts=json.load(f)
print("ideal: ", ideal_noise_est_counts)
print("noise est dist: ", noisy_noise_est_dist)
print("noisy counts: ", noisy_counts)
print("ideal counts: ", ideal_counts)
col_label=list(dict(ideal_noise_est_counts).keys())[0]
# corrected_counts=ne_ult.get_corrected_counts_from_counts(noisy_counts, noisy_noise_est_dist, list(ideal_noise_est_counts.keys())[0])

# print(corrected_counts)

ideal:  {'0010111100': 200000}
noise est dist:  {'1110111010': 3e-05, '0110101111': 1e-05, '0000100100': 8.5e-05, '0000111100': 0.01284, '0110111100': 0.01769, '0100111100': 0.004415, '0000101100': 0.000205, '0000110100': 0.00022, '1010111100': 0.009805, '0010111100': 0.827775, '0011111101': 6.5e-05, '0010111000': 0.01272, '0001111100': 0.00418, '1110010100': 1e-05, '1010011000': 1e-05, '0011111110': 0.000215, '0110011100': 0.00031, '0010101100': 0.01341, '0011111100': 0.013555, '0011011101': 4e-05, '1011010100': 5e-06, '0010011100': 0.013195, '0010110000': 0.004505, '0001010100': 1e-05, '0010111010': 0.004485, '0010001100': 0.004305, '1110111100': 0.00642, '1110111000': 0.000125, '0010100100': 0.004295, '0000100010': 5e-06, '0000111110': 0.000165, '0010111110': 0.010905, '1010011110': 5e-06, '0011111000': 0.000235, '0010111101': 0.00366, '0010111111': 0.002165, '1011011100': 4e-05, '0010010100': 0.00024, '1100111100': 0.00019, '0001001100': 2e-05, '0001110100': 6.5e-05, '1110011100': 

In [33]:
# plot_histogram([ideal_counts, noisy_counts, corrected_counts], number_to_keep=5, legend=["ideal counts", "noisy counts", "corrected counts"], figsize=(20,20), filename="plot_sim.jpg")


In [34]:
num_qubits=len(list(noisy_counts.items())[0][0])

if 2**15<2**num_qubits:
    num_keep=2**10#num_qubits#2**10
else:
    num_keep=2**num_qubits
noisy_counts_orig=noisy_counts
print("num keep ", num_keep)
noisy_noise_est_dist=dist_ult.convert_to_first_col(noisy_noise_est_dist, col_label)
noisy_noise_est_dist=dist_ult.filter_counts(noisy_noise_est_dist, num_keep)
noisy_counts=dist_ult.filter_counts(noisy_counts, num_keep)
all_keys, noisy_noise_est_dist, noisy_counts=dist_ult.fill_counts_range(noisy_noise_est_dist, noisy_counts, num_keep, num_qubits)
print(noisy_counts)
print(noisy_noise_est_dist)

noisy_noise_est_vec=sorted(list(noisy_noise_est_dist.items()), key=lambda x: int(x[0], 2))
noisy_noise_est_vec=[elem[1] for elem in noisy_noise_est_vec]
counts_vec=sorted(list(noisy_counts.items()), key=lambda x: int(x[0], 2))
counts_vec=[elem[1] for elem in counts_vec]
# print(len(noisy_noise_est_vec))
# print(len(counts_vec))
quasi_corrected_counts=dist_ult.get_quasi_corrected(noisy_noise_est_vec, counts_vec)
quasi_corrected_counts=dist_ult.label_quasis(quasi_corrected_counts, all_keys)
# print(quasi_corrected_counts)
quasi_method=QuasiDistribution(quasi_corrected_counts)
quasi_corrected_counts=quasi_method.nearest_probability_distribution().binary_probabilities(num_qubits)
# print(quasi_corrected_counts)
# quasi_corrected_counts={bin(idx)[2::].zfill(num_qubits):val for idx, val in enumerate(quasi_corrected_counts)}



num keep  1024
max exist  976
min exist  0
size before  397
{'0000000000': 5239, '0000000001': 17126, '0000000010': 17404, '0000000011': 236, '0000000100': 17326, '0000000101': 247, '0000000110': 265, '0000000111': 17, '0000001000': 17479, '0000001001': 253, '0000001010': 292, '0000001011': 23, '0000001100': 313, '0000001101': 25, '0000001110': 16, '0000001111': 0, '0000010000': 17229, '0000010001': 285, '0000010010': 359, '0000010011': 26, '0000010100': 324, '0000010101': 22, '0000010110': 28, '0000010111': 0, '0000011000': 289, '0000011001': 23, '0000011010': 32, '0000011011': 0, '0000011100': 22, '0000011101': 0, '0000011110': 1, '0000011111': 0, '0000100000': 17142, '0000100001': 288, '0000100010': 331, '0000100011': 19, '0000100100': 334, '0000100101': 26, '0000100110': 33, '0000100111': 0, '0000101000': 319, '0000101001': 34, '0000101010': 35, '0000101011': 0, '0000101100': 28, '0000101101': 1, '0000101110': 0, '0000101111': 0, '0000110000': 296, '0000110001': 30, '0000110010': 3

In [35]:
fig=plot_histogram([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=15, #bar_labels=False,
               legend=["ideal", "raw", "corrected"], title="Dicke n=20, k=1 Simulation", bar_labels=False)
fig.savefig("plot_dicke20_1_counts_sim.jpg", bbox_inches="tight")

In [36]:
fig=plot_distribution([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=15, #bar_labels=False,
                  legend=["ideal", "raw", "corrected"], title="Dicke n=20, k=1 Simulation", bar_labels=False) #figsize=(15,15))
ax=fig.axes[0]
ax.set_ylabel("Probability")
fig.tight_layout()
fig.savefig("plot_dicke20_1_dist_sim.jpg", bbox_inches="tight")